In [1]:
#import the necessary libraries
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import requests
import numpy as np
import math
from math import radians, sin, cos, sqrt, atan2
import xarray
import rioxarray
import matplotlib.pyplot as plt
import itertools

In [2]:
#Create a function that handles the daily scraping of our source URL

#list of states by abbrv. - the lowercase state abbrv. are used in the url that we scrape from
states = ['AL','AK','AZ','AR', 'CA','CO','CT','DE','FL','GA','HI','ID','IL','IN',
'IA','KS','KY','LA','ME','MD','MA','MI','MN','MS','MO','MT','NE','NV','NH','NJ','NM','NY',
'NC','ND','OH','OK','OR','PA','RI','SC','SD','TN','TX','UT','VT','VA','WA','WV','WI','WY']
states = list(map(str.lower,states))

In [3]:
#Instantiate the driver and set our starting url
driver=webdriver.Chrome()

In [4]:
#function which scrapes current state of all of our USGS stations available with flood state information

def TableScrape():
    url = 'https://waterwatch.usgs.gov/index.php?id=flood&sid=w__table&r=ri'
    df = pd.DataFrame()
    for x in range(len(states)):
        url = url[:-2]+states[x]
        driver.get(url)
        hold_df = pd.read_html(url)[4:]
        hold_df = hold_df[0]
        df = df.append(hold_df, ignore_index=True)
    
    return df
        

In [5]:
#Instantiate a dataframe that contains all of our USGS flooding information
df = TableScrape()

C:\Users\ASKhazami\AppData\Local\Temp\ipykernel_13448\2734864620.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(hold_df, ignore_index=True)
C:\Users\ASKhazami\AppData\Local\Temp\ipykernel_13448\2734864620.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(hold_df, ignore_index=True)
C:\Users\ASKhazami\AppData\Local\Temp\ipykernel_13448\2734864620.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(hold_df, ignore_index=True)
C:\Users\ASKhazami\AppData\Local\Temp\ipykernel_13448\2734864620.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(hold_df, ignore_index=True)


C:\Users\ASKhazami\AppData\Local\Temp\ipykernel_13448\2734864620.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(hold_df, ignore_index=True)
C:\Users\ASKhazami\AppData\Local\Temp\ipykernel_13448\2734864620.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(hold_df, ignore_index=True)
C:\Users\ASKhazami\AppData\Local\Temp\ipykernel_13448\2734864620.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(hold_df, ignore_index=True)
C:\Users\ASKhazami\AppData\Local\Temp\ipykernel_13448\2734864620.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(hold_df, ignore_index=True)


In [6]:
# write all of our functions to scrape USGS gauge information at a more granular level
# for streamflow, surface water level, ground water level, spring water level, and precipitation

# dicts containing all the information that pertains to our percipitation data from USGS map
# meaning that this is the network data that is called from https://dashboard.waterdata.usgs.gov/app/nwd/en/?region=lower48&aoi=default
# the value in question for these gauges is percipitation - total inches

def getPrecipitation():
    headers_precip = {
        'Accept': '*/*',
        'Accept-Language': 'en-US,en;q=0.9',
        'Connection': 'keep-alive',
        # 'Cookie': '_hjSessionUser_606685=eyJpZCI6Ijg0MTMwZDAyLWJjZTYtNTYwMC1hNzhlLTUxNDA2ZGI0YWY1NyIsImNyZWF0ZWQiOjE2ODM3MzMyNTg5MzUsImV4aXN0aW5nIjp0cnVlfQ==; _ga=GA1.3.2126571673.1683733259; _ga=GA1.4.2126571673.1683733259; __utmc=191289900; _gid=GA1.2.1982434073.1684175396; _ga_Q3BG15MFSR=GS1.1.1684253524.4.0.1684253528.0.0.0; _ga_BTG7QB0V1W=GS1.1.1684338231.2.0.1684338231.0.0.0; _gid=GA1.4.1982434073.1684175396; __utma=191289900.2126571673.1683733259.1684427918.1684429964.7; __utmz=191289900.1684429964.7.5.utmcsr=dashboard.waterdata.usgs.gov|utmccn=(referral)|utmcmd=referral|utmcct=/; _ga_232G9MDQL1=GS1.1.1684507551.13.0.1684507553.0.0.0; _ga=GA1.2.2126571673.1683733259; _gat_UA-179298358-1=1',
        'Referer': 'https://dashboard.waterdata.usgs.gov/app/nwd/en/?region=lower48&aoi=default',
        'Sec-Fetch-Dest': 'empty',
        'Sec-Fetch-Mode': 'cors',
        'Sec-Fetch-Site': 'same-origin',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36',
        'sec-ch-ua': '"Chromium";v="112", "Google Chrome";v="112", "Not:A-Brand";v="99"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"Windows"',
    }

    response_precipitation = requests.get(
        'https://dashboard.waterdata.usgs.gov/service/cwis/1.0/odata/CurrentConditions?$top=15000&$filter=(AccessLevelCode%20eq%20%27P%27)%20and%20(1%20eq%201%20and%20true)%20and%20(ParameterCode%20in%20(%2772192%27%2C%2772194%27%2C%2799772%27%2C%2700045%27%2C%2700193%27))&$select=AgencyCode,SiteNumber,SiteName,SiteTypeCode,Latitude,Longitude,CurrentConditionID,ParameterCode,TimeLocal,TimeZoneCode,Value,ValueFlagCode,RateOfChangeUnitPerHour,StatisticStatusCode,FloodStageStatusCode&$orderby=SiteNumber,AgencyCode,ParameterCode,TimeLocal%20desc&caller=National%20Water%20Dashboard%20default',
        headers=headers_precip,
    )
    
    return response_precipitation.json()['value']

# dicts containing all the information that pertains to our spring water level data from USGS map
# meaning that this is the network data that is called from https://dashboard.waterdata.usgs.gov/app/nwd/en/?region=lower48&aoi=default
# the value in question for these is gauge height 
def getSpringWaterLevel():
    headers_spring = {
        'Accept': '*/*',
        'Accept-Language': 'en-US,en;q=0.9',
        'Connection': 'keep-alive',
        'Referer': 'https://dashboard.waterdata.usgs.gov/app/nwd/en/?region=lower48&aoi=default',
        'Sec-Fetch-Dest': 'empty',
        'Sec-Fetch-Mode': 'cors',
        'Sec-Fetch-Site': 'same-origin',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36',
        'sec-ch-ua': '"Chromium";v="112", "Google Chrome";v="112", "Not:A-Brand";v="99"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"Windows"',
    }

    response_spring = requests.get(
        'https://dashboard.waterdata.usgs.gov/service/cwis/1.0/odata/CurrentConditions?$top=15000&$filter=(AccessLevelCode%20eq%20%27P%27)%20and%20(1%20eq%201%20and%20true)%20and%20(SiteTypeCode%20in%20(%27SP%27))%20and%20(ParameterCode%20in%20(%2730207%27%2C%2730210%27%2C%2730211%27%2C%2730212%27%2C%2730213%27%2C%2762600%27%2C%2762601%27%2C%2762610%27%2C%2762611%27%2C%2762612%27%2C%2762613%27%2C%2762614%27%2C%2762615%27%2C%2762616%27%2C%2762617%27%2C%2762618%27%2C%2762619%27%2C%2762620%27%2C%2762621%27%2C%2762622%27%2C%2762623%27%2C%2762624%27%2C%2763158%27%2C%2763159%27%2C%2763160%27%2C%2763161%27%2C%2772019%27%2C%2772020%27%2C%2772150%27%2C%2772170%27%2C%2772171%27%2C%2772178%27%2C%2772199%27%2C%2772214%27%2C%2772215%27%2C%2772226%27%2C%2772227%27%2C%2772228%27%2C%2772229%27%2C%2772230%27%2C%2772231%27%2C%2772232%27%2C%2772251%27%2C%2772264%27%2C%2772265%27%2C%2772275%27%2C%2772276%27%2C%2772279%27%2C%2772292%27%2C%2772293%27%2C%2772335%27%2C%2772336%27%2C%2799019%27%2C%2799020%27%2C%2799065%27%2C%2799227%27%2C%2700062%27%2C%2700065%27%2C%2700072%27))&$select=AgencyCode,SiteNumber,SiteName,SiteTypeCode,Latitude,Longitude,CurrentConditionID,ParameterCode,TimeLocal,TimeZoneCode,Value,ValueFlagCode,RateOfChangeUnitPerHour,StatisticStatusCode,FloodStageStatusCode&$orderby=SiteNumber,AgencyCode,ParameterCode,TimeLocal%20desc&caller=National%20Water%20Dashboard%20default',
        headers=headers_spring,
    )
    
    return response_spring.json()['value']

# dicts containing all the information that pertains to our ground water level data from USGS map
# meaning that this is the network data that is called from https://dashboard.waterdata.usgs.gov/app/nwd/en/?region=lower48&aoi=default
# the value in question for these is depth to water - feet below land surface
def getGroundWaterLevel():
    headers_gwl = {
        'Accept': '*/*',
        'Accept-Language': 'en-US,en;q=0.9',
        'Connection': 'keep-alive',
        'Referer': 'https://dashboard.waterdata.usgs.gov/app/nwd/en/?region=lower48&aoi=default',
        'Sec-Fetch-Dest': 'empty',
        'Sec-Fetch-Mode': 'cors',
        'Sec-Fetch-Site': 'same-origin',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36',
        'sec-ch-ua': '"Chromium";v="112", "Google Chrome";v="112", "Not:A-Brand";v="99"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"Windows"',
    }

    response_gwl = requests.get(
        'https://dashboard.waterdata.usgs.gov/service/cwis/1.0/odata/CurrentConditions?$top=15000&$filter=(AccessLevelCode%20eq%20%27P%27)%20and%20(1%20eq%201%20and%20true)%20and%20(SiteTypeCode%20in%20(%27GW%27%2C%27GW-CR%27%2C%27GW-EX%27%2C%27GW-HZ%27%2C%27GW-IW%27%2C%27GW-MW%27%2C%27GW-TH%27%2C%27SB%27%2C%27SB-CV%27%2C%27SB-GWD%27%2C%27SB-TSM%27%2C%27SB-UZ%27))%20and%20(ParameterCode%20in%20(%2730207%27%2C%2730210%27%2C%2730211%27%2C%2730212%27%2C%2730213%27%2C%2762600%27%2C%2762601%27%2C%2762610%27%2C%2762611%27%2C%2762612%27%2C%2762613%27%2C%2762614%27%2C%2762615%27%2C%2762616%27%2C%2762617%27%2C%2762618%27%2C%2762619%27%2C%2762620%27%2C%2762621%27%2C%2762622%27%2C%2762623%27%2C%2762624%27%2C%2763158%27%2C%2763159%27%2C%2763160%27%2C%2763161%27%2C%2772019%27%2C%2772020%27%2C%2772150%27%2C%2772170%27%2C%2772171%27%2C%2772178%27%2C%2772199%27%2C%2772214%27%2C%2772215%27%2C%2772226%27%2C%2772227%27%2C%2772228%27%2C%2772229%27%2C%2772230%27%2C%2772231%27%2C%2772232%27%2C%2772251%27%2C%2772264%27%2C%2772265%27%2C%2772275%27%2C%2772276%27%2C%2772279%27%2C%2772292%27%2C%2772293%27%2C%2772335%27%2C%2772336%27%2C%2799019%27%2C%2799020%27%2C%2799065%27%2C%2799227%27%2C%2700062%27%2C%2700065%27%2C%2700072%27))&$select=AgencyCode,SiteNumber,SiteName,SiteTypeCode,Latitude,Longitude,CurrentConditionID,ParameterCode,TimeLocal,TimeZoneCode,Value,ValueFlagCode,RateOfChangeUnitPerHour,StatisticStatusCode,FloodStageStatusCode&$orderby=SiteNumber,AgencyCode,ParameterCode,TimeLocal%20desc&caller=National%20Water%20Dashboard%20default',
        headers=headers_gwl,
    )
    
    return response_gwl.json()['value']

# dicts containing all the information that pertains to our surface water level data from USGS map
# meaning that this is the network data that is called from https://dashboard.waterdata.usgs.gov/app/nwd/en/?region=lower48&aoi=default
# This is all of the surface water level data that exists for USGS - the value that it refers to is gauge height level
def getSurfaceWaterLevel():
    headers_swl = {
        'Accept': '*/*',
        'Accept-Language': 'en-US,en;q=0.9',
        'Connection': 'keep-alive',
        'Referer': 'https://dashboard.waterdata.usgs.gov/app/nwd/en/?region=lower48&aoi=default',
        'Sec-Fetch-Dest': 'empty',
        'Sec-Fetch-Mode': 'cors',
        'Sec-Fetch-Site': 'same-origin',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36',
        'sec-ch-ua': '"Chromium";v="112", "Google Chrome";v="112", "Not:A-Brand";v="99"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"Windows"',
    }

    response_swl = requests.get(
        'https://dashboard.waterdata.usgs.gov/service/cwis/1.0/odata/CurrentConditions?$top=15000&$filter=(AccessLevelCode%20eq%20%27P%27)%20and%20(1%20eq%201%20and%20true)%20and%20(SiteTypeCode%20in%20(%27ES%27%2C%27LK%27%2C%27OC%27%2C%27OC-CO%27%2C%27ST%27%2C%27ST-CA%27%2C%27ST-DCH%27%2C%27ST-TS%27%2C%27WE%27))%20and%20(ParameterCode%20in%20(%2700065%27))&$select=AgencyCode,SiteNumber,SiteName,SiteTypeCode,Latitude,Longitude,CurrentConditionID,ParameterCode,TimeLocal,TimeZoneCode,Value,ValueFlagCode,RateOfChangeUnitPerHour,StatisticStatusCode,FloodStageStatusCode&$orderby=SiteNumber,AgencyCode,ParameterCode,TimeLocal%20desc&caller=National%20Water%20Dashboard%20default',
        headers=headers_swl,
    )
    
    return response_swl.json()['value']

# Here we are scraping all of the stream flow data from
# https://dashboard.waterdata.usgs.gov/app/nwd/en/?region=lower48&aoi=default
# the value we receive is discharge in cubic ft/s
def getStreamFlow():
    headers = {
        'Accept': '*/*',
        'Accept-Language': 'en-US,en;q=0.9',
        'Connection': 'keep-alive',
        'Referer': 'https://dashboard.waterdata.usgs.gov/app/nwd/en/?region=lower48&aoi=default',
        'Sec-Fetch-Dest': 'empty',
        'Sec-Fetch-Mode': 'cors',
        'Sec-Fetch-Site': 'same-origin',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36',
        'sec-ch-ua': '"Chromium";v="112", "Google Chrome";v="112", "Not:A-Brand";v="99"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"Windows"',
    }

    response_streamflow = requests.get(
        'https://dashboard.waterdata.usgs.gov/service/cwis/1.0/odata/CurrentConditions?$top=15000&$filter=(AccessLevelCode%20eq%20%27P%27)%20and%20(1%20eq%201%20and%20true)%20and%20(SiteTypeCode%20in%20(%27ST%27%2C%27ST-CA%27%2C%27ST-DCH%27%2C%27ST-TS%27))%20and%20(ParameterCode%20in%20(%2730208%27%2C%2730209%27%2C%2750042%27%2C%2750050%27%2C%2750051%27%2C%2772137%27%2C%2772138%27%2C%2772139%27%2C%2772177%27%2C%2772243%27%2C%2774072%27%2C%2781395%27%2C%2799060%27%2C%2799061%27%2C%2700056%27%2C%2700058%27%2C%2700059%27%2C%2700060%27%2C%2700061%27))&$select=AgencyCode,SiteNumber,SiteName,SiteTypeCode,Latitude,Longitude,CurrentConditionID,ParameterCode,TimeLocal,TimeZoneCode,Value,ValueFlagCode,RateOfChangeUnitPerHour,StatisticStatusCode,FloodStageStatusCode&$orderby=SiteNumber,AgencyCode,ParameterCode,TimeLocal%20desc&caller=National%20Water%20Dashboard%20default',
        headers=headers,
    )
    
    return response_streamflow.json()['value']

In [7]:
#Convert all of our json responses from the webpages into dataframes
#make some adjustments to column headers for clarity
#more specifically - we want to specify what the values represent for each scraped 
#dataframe and what the rate of change represents

swl_df = pd.DataFrame.from_dict(getSurfaceWaterLevel())
streamflow_df = pd.DataFrame.from_dict(getStreamFlow())
gwl_df = pd.DataFrame.from_dict(getGroundWaterLevel())
spring_df = pd.DataFrame.from_dict(getSpringWaterLevel())
precip_df = pd.DataFrame.from_dict(getPrecipitation())
streamflow_df.rename(columns={'Value':'Discharge(ft^3/s)', 'RateOfChangeUnitPerHour':'RateofChange_Discharge(ft^3/s)'}, inplace=True)
swl_df.rename(columns={'Value':'Gauge Height(ft)', 'RateOfChangeUnitPerHour':'RateofChange_Gauge(ft)'},inplace=True)
gwl_df.rename(columns={'Value':'Depth to Water Below Land Surface(ft)', 'RateOfChangeUnitPerHour':'RateOfChange_groundwaterlevel(ft)'}, inplace=True)
spring_df.rename(columns={'Value':'Gauge Height Spring(ft)', 'RateOfChangeUnitPerHour':'RateOfChange_springwaterlevel(ft)'}, inplace=True)
precip_df.rename(columns={'Value':'Precipitation(in)', 'RateOfChangeUnitPerHour':'RateOfChangeUnitPerHour_precip(in)'}, inplace=True)


In [8]:
#Create a method which combines all of the available data into one single source dataframe
#This returns a dataframe that has all the characteristics available for each gauge
#We can also combine the floodstage information so we know

def DataMerge(streamflow,spring,surface,precipitation,ground):
    merge_df = streamflow.combine_first(spring).combine_first(surface).combine_first(precipitation).combine_first(ground)
    return merge_df

#Make our merged data frame by calling DataMerge function - for all our non-flood status data at first
merge = DataMerge(streamflow_df, spring_df, swl_df, precip_df, gwl_df)

In [9]:
#if USGS Station name is only 7-digits long, then during parsing read_html cuts off the leading 0 and cast it as
#an integer. We need to add that leading zero back on so our merge table and our flood stage table align

df['USGSstationID'] = ['0' + str(value) if len(str(value)) == 7 else str(value) for value in df['USGSstationID']]
df.rename(columns={'USGSstationID':'SiteNumber'}, inplace=True)

#Now merge our dataframes on SiteNumber
merge_final = pd.merge(merge,df,on='SiteNumber',how='outer')

#We can drop all of the locations where merge_final does not have long/lat as we cannot use these
merge_final = merge_final[(~merge_final['Latitude'].isna())&(~merge_final['Longitude'].isna())]

In [10]:
#We will create a SQLite3 database here so we can continually update the data we have with the most recent data
#For instance, if we have a USGS gauge that is registering that it is in a flood stage - but there is no NWS
#data that indicates what the flood stage for that particular part of the river is, then we will update it that value
#with the current stage and streamflow (can also include some other parameters down the line if necessary)

import sqlite3

#The data frame that we are going to use for this is the merge_final data frame which is a collection of 
#USGS gauge data as well as NWS data

db_file = 'USGS_update.db'
conn = sqlite3.connect(db_file)

#Create the dataframe that will house all of the historical data and current thresholds for 
#determining flood stage

#Create a database that we can draw from that contains flood stage statuses
hist_flood_ref_df = merge_final

hist_flood_ref_df.to_sql('USGS_hist_data', conn, if_exists='replace', index=False)

conn.close()

In [11]:
#Test to make sure our sqlite tbl is converting properly and appending
conn = sqlite3.connect('USGS_update.db')
z = pd.read_sql_query("SELECT * FROM USGS_hist_data", conn)

In [12]:
#Our flood risk adjustments include removing all USGS gauges that are in 'NOFLOOD', 'None', N/A
#this narrows our scope to areas that are being reported as flooding
floodRisk_df = merge_final[((merge_final['FloodStageStatusCode']!='NOFLOOD')|
                            (merge_final['FloodStageStatusCode']!='None')|(~merge_final['FloodStageStatusCode'].isna()))
                           |(~merge_final['NWSfloodstage(ft)'].isna())]

In [16]:
#This is National Water Model data and other geographic data found which will help fine tune our model
fn_river = 'RouteLink_CONUS.nc' 
fn_river2 = 'newNWM.nc'
fn_long_range = 'Fulldom_CONUS_LongRange.nc'

#Opening NWM data using xarray - netCDF files should be opened using xarray
data_river = xarray.open_dataset(fn_river)
data_river2 = xarray.open_dataset(fn_river2)
data_lr = xarray.open_dataset(fn_long_range)

In [17]:
#This is the streamflow and node information coerced into a dataframe
df_river = data_river.to_dataframe()
df_river2 = data_river2.to_dataframe()

#long range is data that has ~1km mesh to between elevation data, this will be our baseline for estimating the
#flooding based on change from river elevation to nearest structure elevation
long_range_df = data_lr.to_dataframe()

In [15]:
#We want to only retain all of the long_range data that is pertinent to us
#We drop all the N/A where there is no TOPOGRAPHY or FLOWDIRECTION data
#We also found through ARCGIS testing that if STREAMORDER == -9999.0 it is referencing non-river locations
long_range_df = long_range_df[~long_range_df['FLOWDIRECTION'].isna()]
long_range_df = long_range_df[~long_range_df['TOPOGRAPHY'].isna()]
long_range_df = long_range_df[long_range_df['STREAMORDER'] != -9999.0]

In [18]:
#We merge our NWM and other river information
#This is the connection between our NWM data and geographic feature information
merge_NWM = pd.merge(df_river, df_river2, left_on='link', right_on='feature_id', how='outer')

In [19]:
#import our elevation data into a usable dataset
elevation_data = pd.read_csv('Louisiana_Stucture_Altitude.csv')

#Remove the Nan values from our data
elevation_data = elevation_data[~elevation_data['gridcode'].isna()]
elevation_data = elevation_data.reset_index()

In [20]:
#Function to find the nearest structures --> first to USGS flood stage warnings
#This function is used to find the nearest structure to our USGS flood gauge
from scipy.spatial import KDTree
import geopy.distance

def NearestStruct():
    #Get our longitude and latitude values for our structures and our gauges
    coords_flood = floodRisk_df[['Latitude','Longitude']].values
    coords_struct = elevation_data[['latitude','longitude']].values
    #Create a KDTree (used for efficiently finding the nearest neighbors to a point)
    kdt = KDTree(coords_struct)
    k_nearest = []
    for i, row in floodRisk_df.iterrows():
        gauge_point = row[['Latitude','Longitude']].values
        _, indices = kdt.query(gauge_point, k=1)
        nearest_struct = coords_struct[indices]
        distance = geopy.distance.distance(gauge_point, nearest_struct).miles
        struct = nearest_struct.flatten().tolist()
        struct.append(distance)
        struct.extend(list(gauge_point))
        k_nearest.append(struct)
    #Create our dataframe containing our coordinates and the distance between nearest building and gauge
    #but only if the building is within 3 miles of the gauge
    columns = ['Lat_nearest_struct', 'Long_nearest_struct', 'Distance(miles)_nearest_gauge', 'Lat_gauge', 'Long_gauge']
    df_nearest = pd.DataFrame(k_nearest, columns=columns)
    df_nearest = df_nearest[df_nearest['Distance(miles)_nearest_gauge'] < 3].reset_index()
    
    return df_nearest

In [21]:
#We can now run our nearest structure function and assign the dataframe output
#Since for now all of our elevation data is from Louisiana - all of the nearest structures will be within LA
dfx = NearestStruct()

In [22]:
#our floodrisk_df contains all of our data for the gauges that currently read some level of flooding
#our dfx dataframe contains the nearest location to all of the gauges in the US that read flooding,
#however in our NearestStructure() definition we set the range of flood risk gauges to nearest structure
#to 5000m so the furthest ourside of LA a gauge can possibly be is around 3miles
#combined_df gives us the USGS information combined with the distance information we need
combined_df = pd.merge(dfx, floodRisk_df, left_on='Lat_gauge', right_on='Latitude', how='left')

In [23]:
#Here we are only keeping the combined rows that have a Flood status warnings 
#Check why this doesn't retain the edits made by floodrisk_df?
combined_df = combined_df[((combined_df['FloodStageStatusCode']!='NOFLOOD')&
                            (combined_df['FloodStageStatusCode']!='None')&(~combined_df['FloodStageStatusCode'].isna())
                  &(combined_df['FloodStageStatusCode']!='NA'))]

In [24]:
combined_df

,index,Lat_nearest_struct,Long_nearest_struct,Distance(miles)_nearest_gauge,Lat_gauge,Long_gauge,AgencyCode,CurrentConditionID,Depth to Water Below Land Surface(ft),Discharge(ft^3/s),...,TimeLocal,TimeZoneCode,ValueFlagCode,USGS station name,NWSfloodstage(ft),Most recentstage(ft),Most recentflow(cfs),Historicalpeaks(cfs),Most recentlocaldate/time,NWSstationID
10,5805,32.994423,-93.407545,0.638043,32.994581,-93.396560,USGS,62249.0,NaN,1210.0,...,2023-06-15T14:00:00Z,CDT,NaN,"Bayou Dorcheat near Springhill, LA",11.0,11.88,1220.0,36700.0,2023-06-15 13:00:00,SPHL1
59,5885,29.693108,-91.208738,0.193392,29.692819,-91.211937,USGS,167820.0,NaN,59900.0,...,2023-06-15T14:00:00Z,CDT,NaN,"Lower Atchafalaya River at Morgan City, LA",6.0,4.13,52400.0,512000.0,2023-06-15 13:00:00,MCGL1
63,5889,30.071343,-91.826405,0.173757,30.071035,-91.829284,USGS,278558.0,NaN,337.0,...,2023-06-15T13:45:00Z,CDT,NaN,Bayou Teche at Keystone L&D nr St. Martinville...,NaN,10.13,345.0,4430.0,2023-06-15 12:45:00,NaN


In [25]:
#For each of our remaining USGS gauges (those which indicate some level of flooding) - we need 
#to create boudning boxes so that we can capture all of the NWM nodes that fall within that range
def create_bounding_boxes(lat,lon,distance):
    distance_meters = distance*1609.34
    #latitude and longitude difference in meters
    lat_diff = distance_meters/111111
    lon_diff = distance_meters/(111111 * abs(math.cos(math.radians(lat))))
    
    #bounding boxes
    lat_min = lat-lat_diff
    lat_max = lat+lat_diff
    lon_min = lon-lon_diff
    lon_max = lon+lon_diff
    
    return [lat_min, lat_max, lon_min, lon_max]

In [26]:
#Here we actually generate all of the bouding box coordinates using our function above
#This is for all of the USGS gauges of interest (flooding status indicated)
#Note that this ONLY creates the bounding box coordinates and appends them as columns to combined_df

#Create the columns to hold the bouding boxes
combined_df['1mile']=None
combined_df['3mile']=None
combined_df['5mile']=None

for index, row in combined_df.iterrows():
    #Get the latitude and longitude we want to create our bounding boxes around (gauge lat/lon)
    lat = row['Lat_gauge']
    lon = row['Long_gauge']
    
    #Create the bouding boxes for each of the different 1,3,5 mile ranges we want
    bbox_1mile = create_bounding_boxes(lat,lon,1)
    bbox_3mile = create_bounding_boxes(lat,lon,3)
    bbox_5mile = create_bounding_boxes(lat,lon,5)

    #Append bouding boxes
    combined_df.at[index,'1mile'] = bbox_1mile
    combined_df.at[index,'3mile'] = bbox_3mile
    combined_df.at[index,'5mile'] = bbox_5mile

In [27]:
#Now we can create our get sub-dataframes from our NWM dataframe and append these onto our data frame
#This is where we use the bounding boxes that we made and create actual dataframes that hold the relevant informatio
#for instance we create dataframes that contain all of NWM points that fall within our bounding box
#This can be converted into a function for greater abstraction

#Create the columns which will house our in-radius NWM data
combined_df['within_1mile_df'] = None
combined_df['within_3mile_df'] = None
combined_df['within_5mile_df'] = None

#This is the in range long range data
combined_df['within_1mile_lr_df'] = None
combined_df['within_3mile_lr_df'] = None
combined_df['within_5mile_lr_df'] = None


for index, row in combined_df.iterrows():
    
    #Get the NWM nodes that fall within 1,3,5 mile radii that we have established around our USGS gauges
    #First get our 1,3,5 mile radii for each of the corresponding rows in our in dataframe
    lat_lon_1mile = create_bounding_boxes(row['Lat_gauge'], row['Long_gauge'], 1)
    lat_lon_3mile = create_bounding_boxes(row['Lat_gauge'], row['Long_gauge'], 3)
    lat_lon_5mile = create_bounding_boxes(row['Lat_gauge'], row['Long_gauge'], 5)
    
    ##################################################################################Turn into functions#########
    #Create a df for each of our rows that contains only the NWM nodes within the radii
    one_mile_df = merge_NWM[(merge_NWM['lat'] >= lat_lon_1mile[0])
                            &(merge_NWM['lat'] <= lat_lon_1mile[1])
                            &(merge_NWM['lon'] >= lat_lon_1mile[2])
                            &(merge_NWM['lon'] <= lat_lon_1mile[3])]
    three_mile_df = merge_NWM[(merge_NWM['lat'] >= lat_lon_3mile[0])
                            &(merge_NWM['lat'] <= lat_lon_3mile[1])
                            &(merge_NWM['lon'] >= lat_lon_3mile[2])
                            &(merge_NWM['lon'] <= lat_lon_3mile[3])]
    five_mile_df = merge_NWM[(merge_NWM['lat'] >= lat_lon_5mile[0])
                            &(merge_NWM['lat'] <= lat_lon_5mile[1])
                            &(merge_NWM['lon'] >= lat_lon_5mile[2])
                            &(merge_NWM['lon'] <= lat_lon_5mile[3])]
    
    #These dataframes for the long_range data that we pulled
    one_mile_lr_df = long_range_df[(long_range_df['LATITUDE'] >= lat_lon_1mile[0])
                            &(long_range_df['LATITUDE'] <= lat_lon_1mile[1])
                            &(long_range_df['LONGITUDE'] >= lat_lon_1mile[2])
                            &(long_range_df['LONGITUDE'] <= lat_lon_1mile[3])]
    three_mile_lr_df = long_range_df[(long_range_df['LATITUDE'] >= lat_lon_3mile[0])
                            &(long_range_df['LATITUDE'] <= lat_lon_3mile[1])
                            &(long_range_df['LONGITUDE'] >= lat_lon_3mile[2])
                            &(long_range_df['LONGITUDE'] <= lat_lon_3mile[3])]
    five_mile_lr_df = long_range_df[(long_range_df['LATITUDE'] >= lat_lon_5mile[0])
                            &(long_range_df['LATITUDE'] <= lat_lon_5mile[1])
                            &(long_range_df['LONGITUDE'] >= lat_lon_5mile[2])
                            &(long_range_df['LONGITUDE'] <= lat_lon_5mile[3])]
    
    #Append our newly created dataframes onto our existing combined_df 
    combined_df.at[index,'within_1mile_df'] = one_mile_df
    combined_df.at[index,'within_3mile_df'] = three_mile_df
    combined_df.at[index,'within_5mile_df'] = five_mile_df
    
    #Append our newly created dataframes for the long range data onto our exisitng combined_df
    combined_df.at[index,'within_1mile_lr_df'] = one_mile_lr_df
    combined_df.at[index,'within_3mile_lr_df'] = three_mile_lr_df
    combined_df.at[index,'within_5mile_lr_df'] = five_mile_lr_df

In [33]:
#We now have a combined_df that has a column containing a dataframe of all relevant NWM and long_range data
#for each of the USGS gauges within a 1,3,5 miles radii

#This is where we will need to make a loop for every single USGS gauge in with an active flood status stage code.
#for now our test_df is using only the 5 mile radius NWM nodes and long range data for one of the USGS gauges
test_df = combined_df['within_5mile_df'].iloc[0]

In [34]:
test_df

,link,from,to,lon,lat,alt,order,Qi,MusK,MusX,...,nCC,TopWdthCC,TopWdth,crs,streamflow,nudge,velocity,qSfcLatRunoff,qBucket,qBtmVertRunoff
1395955,19926733,0,19926747,-93.473274,33.050419,74.370003,1,0.0,3600.0,0.2,...,0.12,18.338831,6.112943,b'',0.770000,0.00,0.32,0.07684,0.06287,176.595008
1396328,19934295,0,19934259,-93.327240,33.066174,85.260002,1,0.0,3600.0,0.2,...,0.12,9.347944,3.115981,b'',0.000000,0.00,0.00,0.00000,0.00000,0.015000
1396335,19934325,0,19934339,-93.389603,33.062504,71.459999,1,0.0,3600.0,0.2,...,0.12,7.088737,2.362912,b'',0.000000,0.00,0.00,0.00000,0.00000,0.001000
1396337,19934349,0,19934355,-93.421318,33.056179,66.580002,1,0.0,3600.0,0.2,...,0.12,10.514965,3.504988,b'',0.030000,0.00,0.16,0.02725,0.00002,0.249000
1396339,19934353,0,19934427,-93.410103,33.061363,69.989998,1,0.0,3600.0,0.2,...,0.12,8.632882,2.877627,b'',0.000000,0.00,0.07,0.00032,0.00000,0.007000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1544441,19935279,0,19935747,-93.396454,32.999363,54.990002,5,0.0,3600.0,0.2,...,0.10,84.409470,28.136490,b'',16.400000,0.00,0.93,0.00000,0.00002,0.000000
1544480,19935747,0,19935751,-93.395233,32.985126,54.349998,5,0.0,3600.0,0.2,...,0.10,88.102333,29.367445,b'',35.599999,0.12,0.41,0.00000,0.00177,9.275000
1544516,19935751,0,19935759,-93.388824,32.963844,53.849998,5,0.0,3600.0,0.2,...,0.10,88.509064,29.503021,b'',31.169999,0.00,0.15,0.00000,0.00281,27.146001
1544555,19935759,0,19935317,-93.374931,32.942425,53.599998,6,0.0,3600.0,0.2,...,0.10,93.850655,31.283552,b'',38.859999,0.00,0.16,0.00000,0.00020,1.036000


In [35]:
#Reset the index for some upcoming calculations
long_range_df.reset_index()

#We only need the lat, lon and topo from these values so we can get these values alone in a new df
a = long_range_df['LATITUDE'].values
b = long_range_df['LONGITUDE'].values
c = long_range_df['TOPOGRAPHY'].values
longr_df = pd.DataFrame({'lat':a, 'lon':b, 'topo':c})


In [36]:
#This gives us the distance between each NWM node and its closest structure, the structure elevation,
#and the estimated node elevation based on TOPO

#Remember during testing we only have a subsample of the nodes within a 5 miles radius of our flooding USGS gauge

from geopy.distance import geodesic
from scipy.spatial import KDTree

#reset the index
longr_df.reset_index()

# Convert latitude and longitude columns to tuples
test_df['coordinates'] = test_df.apply(lambda row: (row['lat'], row['lon']), axis=1)
elevation_data['coordinates'] = elevation_data.apply(lambda row: (row['latitude'], row['longitude']), axis=1)
longr_df['coordinates'] = longr_df.apply(lambda row: (row['lat'], row['lon']), axis=1)


# Create KDTrees for elevation and long range
points = np.radians(elevation_data[['latitude', 'longitude']].values)
kdtree = KDTree(points)

#long range
points2 = np.radians(longr_df[['lat','lon']].values)
kdtree2 = KDTree(points2)

#Find nearest latitude and longitude in elevation_data and long_range_data for each row in test_df
elevations = []
est_river_elevations = []
distances = []

for _, row in test_df.iterrows():
    lat, lon = np.radians(row['lat']), np.radians(row['lon'])
    _, idx = kdtree.query((lat, lon))
    _, idx2 = kdtree2.query((lat, lon))
    nearest_coords = elevation_data.loc[idx, 'coordinates']
    nearest_coords2 = longr_df.loc[idx2, 'coordinates']
    distance = geodesic(row['coordinates'], nearest_coords).meters
    distance2 = geodesic(row['coordinates'], nearest_coords2).meters
    distances.append(distance)
    elevation = elevation_data.loc[idx, 'gridcode']
    elevations.append(elevation)
    est_river_elevation = longr_df.loc[idx2,'topo']
    est_river_elevations.append(est_river_elevation)


test_df['elevation'] = elevations
test_df['distance'] = distances
test_df['est_river_elevation'] = est_river_elevations

C:\Users\ASKhazami\AppData\Local\Temp\ipykernel_13448\3788006090.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['coordinates'] = test_df.apply(lambda row: (row['lat'], row['lon']), axis=1)
C:\Users\ASKhazami\AppData\Local\Temp\ipykernel_13448\3788006090.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['elevation'] = elevations
C:\Users\ASKhazami\AppData\Local\Temp\ipykernel_13448\3788006090.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Data

In [37]:
#This is to get latitude and longitude as separate columns in our dataframe
def lat(coord):
    latitude,_ = coord
    return latitude
def long(coord):
    _,longitude = coord
    return longitude

test_df['latitude'] = test_df['coordinates'].apply(lat)
test_df['longitude'] = test_df['coordinates'].apply(long)

C:\Users\ASKhazami\AppData\Local\Temp\ipykernel_13448\41765664.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['latitude'] = test_df['coordinates'].apply(lat)
C:\Users\ASKhazami\AppData\Local\Temp\ipykernel_13448\41765664.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['longitude'] = test_df['coordinates'].apply(long)


In [38]:
#Reseting the index here and dropping old index because our old index refers to feature_id which is out of order
#with the dropped rows
test_df = test_df.reset_index()
test_df = test_df.drop(columns=['index'])

In [39]:
# Function to calculate distance between two coordinates using haversine formula
def calculate_distance(lat1, lon1, lat2, lon2):
    R = 6371  # Radius of the Earth in kilometers

    # Convert latitude and longitude to radians
    lat1_rad, lon1_rad, lat2_rad, lon2_rad = map(radians, [lat1, lon1, lat2, lon2])

    # Calculate differences between coordinates
    d_lat = lat2_rad - lat1_rad
    d_lon = lon2_rad - lon1_rad

    # Apply haversine formula
    a = sin(d_lat / 2) ** 2 + cos(lat1_rad) * cos(lat2_rad) * sin(d_lon / 2) ** 2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = R * c

    return distance

# Calculate the distance between consecutive coordinates and store it in a new column
result_list = []
for i in range(len(test_df)-1):
    result = calculate_distance(test_df['latitude'][i], test_df['longitude'][i], test_df['latitude'][i+1],test_df['longitude'][i+1])
    result_list.append(result*1000)

In [40]:
result_list.insert(0, 'N/A')
#convert to meters
test_df['distance_to_next_node'] = result_list

In [41]:
test_df

,link,from,to,lon,lat,alt,order,Qi,MusK,MusX,...,qSfcLatRunoff,qBucket,qBtmVertRunoff,coordinates,elevation,distance,est_river_elevation,latitude,longitude,distance_to_next_node
0,19926733,0,19926747,-93.473274,33.050419,74.370003,1,0.0,3600.0,0.2,...,0.07684,0.06287,176.595008,"(33.050418853759766, -93.47327423095703)",74.0,5420.224484,76.0,33.050419,-93.473274,N/A
1,19934295,0,19934259,-93.327240,33.066174,85.260002,1,0.0,3600.0,0.2,...,0.00000,0.00000,0.015000,"(33.0661735534668, -93.32723999023438)",82.0,7482.941518,80.0,33.066174,-93.327240,13721.836445
2,19934325,0,19934339,-93.389603,33.062504,71.459999,1,0.0,3600.0,0.2,...,0.00000,0.00000,0.001000,"(33.062503814697266, -93.38960266113281)",75.0,7049.689356,74.0,33.062504,-93.389603,5825.751445
3,19934349,0,19934355,-93.421318,33.056179,66.580002,1,0.0,3600.0,0.2,...,0.02725,0.00002,0.249000,"(33.05617904663086, -93.42131805419922)",68.0,6112.030052,61.0,33.056179,-93.421318,3038.176122
4,19934353,0,19934427,-93.410103,33.061363,69.989998,1,0.0,3600.0,0.2,...,0.00032,0.00000,0.007000,"(33.061363220214844, -93.41010284423828)",62.0,6700.753981,63.0,33.061363,-93.410103,1193.614538
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111,19935279,0,19935747,-93.396454,32.999363,54.990002,5,0.0,3600.0,0.2,...,0.00000,0.00002,0.000000,"(32.99936294555664, -93.39645385742188)",75.0,980.457505,61.0,32.999363,-93.396454,145.945818
112,19935747,0,19935751,-93.395233,32.985126,54.349998,5,0.0,3600.0,0.2,...,0.00000,0.00177,9.275000,"(32.98512649536133, -93.39523315429688)",57.0,1015.712074,52.0,32.985126,-93.395233,1587.109611
113,19935751,0,19935759,-93.388824,32.963844,53.849998,5,0.0,3600.0,0.2,...,0.00000,0.00281,27.146001,"(32.963844299316406, -93.38882446289062)",55.0,618.615517,52.0,32.963844,-93.388824,2440.815624
114,19935759,0,19935317,-93.374931,32.942425,53.599998,6,0.0,3600.0,0.2,...,0.00000,0.00020,1.036000,"(32.94242477416992, -93.37493133544922)",66.0,865.605117,55.0,32.942425,-93.374931,2711.660308


In [43]:
#This is where we make our calculations (Assuming topography is in est_river_elevation in m)
#Distance is in meters as well
#so our slope is change in elevation per meter -> which we then convert to degrees for 
#geometry calcs

#Note this is also the simplified calculations that assume both river banks have the same slopes -
#we will work on a more accurate formula down the line

#Note: Channel Slope and Channel Side Slope are not the same, one refers to sides of channel other refers to the 
#slope of the whole channel eg. downwards slope of channel
inland_flood = []
flood_severity_ratio = []

for _,row in test_df.iterrows():
    #To get the area of each of our cross sections we draw some variables from our DF
    top_width = row['TopWdth']
    bottom_width = row['BtmWdth']
    channel_slope = row['ChSlp']
    slope = row['So']
    cc_top_width = row['TopWdthCC']
    
    #Use the trapezoidal channel formula here to calculate our cross-sectional area of flow
    #Reference for this math - Normal Depth, Manning's Equation, and Trapezoidal channels CE331 (hydraulic engineering)
    flow_depth = ((top_width-bottom_width)/2)/channel_slope
    area_cross_section = (bottom_width+channel_slope*flow_depth)*flow_depth
    
    #wetted perimeter is the distance along the inside of our trapezoidal channel that water is in contact with
    #the channel sides and bottom
    w = (1+(channel_slope)**2)**0.5
    wetted_perimeter = bottom_width+(2*flow_depth*w)
    
    #Hydraulic radius is the area divided by the wetted perimeter
    hyd_radius = area_cross_section/wetted_perimeter
    
    #hydraulic depth is the depth average (since it is a trapezoidal channel the depth is not uniform at all places)
    #telling us what the depth would be if it were rectangular channel - We should use cross-sectional depth to 
    hyd_depth = area_cross_section/top_width
    
    #Find area using hyd_depth
    area_hyd_depth = (bottom_width+channel_slope*hyd_depth)*hyd_depth
    
    #We will estimate the overflow area of the compound channel - making the assumption that slope channel slope is the same
    #for both the inner and outer channel
    flow_depth_cc = ((cc_top_width-top_width)/2)/channel_slope
    est_cc_area = (bottom_width*channel_slope*flow_depth_cc)*flow_depth_cc
    cc_hyd_depth = est_cc_area/cc_top_width
    cc_hyd_area = (top_width*channel_slope*hyd_depth)*hyd_depth
    est_tot_area = cc_hyd_area + area_hyd_depth
    
    #Average velocity of flow in our bottom channel cross section
    #Using Mannings equation
    avg_velocity = (1/row['n'])*hyd_radius**(2/3)*slope**(1/2)
    
    #Now we estimate the slope from the edge of body of water to the nearest structure to create a triangle
    #from which we can calculate projected water level in meters
    if(est_tot_area*1 < est_tot_area*avg_velocity*1):
        
        #find the overflow area cross-section
        overflow = est_tot_area*avg_velocity*1 - est_tot_area*1 
        #Get the slope from bank to nearest structure
        #slope is done with a simple rise over run calculation
        riv_build_slope = (row['elevation']-row['est_river_elevation'])/distance
        #**************IMPORTANT****** the elevation on the sides of the river is NOT uniform this needs to be updated
        #to reflect that 
        #get angle of slope in degrees - this currently assumes we have the same elevation on each side of the river 
        #need to update the code to reflect this
        runoff_slope = math.atan(riv_build_slope)

        #find the vertical distance above the compound channel water line our water advances
        #We since the formula for area of our channel is a = (bottom_width + slope*height)*height we use 
        #quadratic formula to find height
        a = cc_top_width
        b = riv_build_slope
        c = -(overflow)
        d = (b**2)-(4*a*c)
        m_above_waterline = (-b+math.sqrt(d))/(2*a)
        
        #Now that we have height we can use a simple angle calculation to find the distance up the waterline
        #that the height corresponds too
        distance_inland = abs(m_above_waterline/math.tan(runoff_slope))
        inland_flood.append(distance_inland)
        flood_severity_ratio.append(abs(distance_inland/row['distance']))     
    else:
        inland_flood.append('N/A')
        flood_severity_ratio.append('N/A')
    

test_df['inland_flood'] = inland_flood
test_df['severity_index'] = flood_severity_ratio

In [ ]:
#The more advanced calculations should include the difference in slope between each side of the river and
#an average change in the thickness of the river channel between nodes so we can determine bottlenecks

#begin by finding the nearest eastern, western, northern and southern buildings


In [ ]:
test_df.to_csv('flooding_numbers.csv')